In [25]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
from imutils import paths
import cv2
import tensorflow as tf
import tensorflow.keras.layers as tfl
IMG_X = 64
IMG_Y = 64

In [26]:

def form_training_data(paths,cls):
    X = []
    y = []
    for imgpath,cl in zip(paths,cls):
        try:
            img_array = cv2.imread(imgpath,cv2.IMREAD_GRAYSCALE)
            img_array = cv2.resize(img_array,(IMG_X,IMG_Y))
            X.append(img_array)
            y.append(cl)
        except:
            pass
    
    return np.array(X).reshape(-1,IMG_X,IMG_Y,1),np.array(y)



In [20]:
imagePaths_test_0 = list(paths.list_images("./datasets/test_folder/0/"))
imagePaths_test_1 = list(paths.list_images("./datasets/test_folder/1/"))
imagePaths_test = imagePaths_test_0 + imagePaths_test_1
test_class = [0]*len(imagePaths_test_0)+[1]*len(imagePaths_test_1)
imagePaths_train_0 = list(paths.list_images("./datasets/train_folder/0/"))
imagePaths_train_1 = list(paths.list_images("./datasets/train_folder/1/"))
imagePaths_train = imagePaths_train_0 + imagePaths_train_1
train_class = [0]*len(imagePaths_train_0)+[1]*len(imagePaths_train_1)

X_train,y_train = form_training_data(imagePaths_train,train_class)
X_test,y_test = form_training_data(imagePaths_test,test_class)

In [22]:
#shuffle the training data
shuffler = np.random.permutation(len(X_train))
X_train = X_train[shuffler]
y_train = y_train[shuffler]

In [38]:
model = tf.keras.Sequential([
    tfl.ZeroPadding2D(padding=3,input_shape = X_train.shape[1:],),
    tfl.Conv2D(64,7,strides=(1,1)),
    tfl.BatchNormalization(axis = 3),
    tfl.ReLU(),
    tfl.MaxPool2D(),
    tfl.Flatten(),
    tfl.Dense(1,activation='sigmoid')
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d_2 (ZeroPaddi  (None, 70, 70, 1)        0         
 ng2D)                                                           
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        3200      
                                                                 
 batch_normalization_2 (Batc  (None, 64, 64, 64)       256       
 hNormalization)                                                 
                                                                 
 re_lu_2 (ReLU)              (None, 64, 64, 64)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                             
                                                      

In [39]:

model.compile(optimizer="adam",loss="categorical_crossentropy",metrics = ['accuracy'])

In [41]:
model.fit(X_train,y_train,epochs=10,batch_size=32,validation_split=0.1)

Epoch 1/10
 7/79 [=>............................] - ETA: 14s - loss: 0.0000e+00 - accuracy: 0.4375

In [37]:
model.evaluate(X_train,y_train)

88/88 [==============================] - 5s 56ms/step - loss: 0.0000e+00 - accuracy: 0.4421


[0.0, 0.4421428442001343]